# Introducción a TensorFlow Extended (TFX)
TensorFlow Extended (TFX) es una plataforma de machine learning de escala productiva provista por Google basada en TensorFlow. Provee de un marco de configuración y bibliotecas compartidas para integrar componentes comunes para definir, ejecutar y monitorear un sistema de machine learning.

Componentes opcionales:
- TensorFlow Serving: Sistema de otorgamiento de modelos de machine learning diseñado para entornos de producción. Facilita desplegar nuevos algoritmos y experimentos, manteniendo la misma arquitectura de servidor y APIs.
![TFX Serving Architecture](images/tfx-serving-architecture.png)
- TensorFlow JS: Librería de machine learning en JavaScript para entrenar y desplegar modelos de machine learning en el browser y con Node.js. Encapsula abstracciones, incluyendo objetos como Tensors y Optimizers. ml5.js es una librería que provee de acceso a algoritmos y modelos de machine learning con un API conciso y fácil de usar. Por medio de TensorFlow.js es posible: programar redes neuronales en JavaScript, convertir modelos pre entrenados en Python a TensorFlow.js, visualizar el comportamiento de modelos TensorFlow.js, preparar datos para procesamiento con TensorFlow.js.
- TensorFlow Lite: Marco de trabajo para deep learning de código abierto orientado a inferencia en dispositivos. Provee de un conjunto de herramientas para que los desarrolladores ejecuten modelos TensorFlow en dispositivos móvil, empotrados e IoT. Permite inferencia machine learning en los dispositivos para mejorar el tamaño de los binarios y el desempeño. Consta de un intérpete que corre modelos especialmente optimizados en diversos tipos de hardware y un convertidor que transforma modelos TensorFlow para ser empleados por el intérprete y que puede introducir optimizaciones para mejorar el tamaño del binario y el desempeño.

# Requisitos
TFX requiere de TensorFlow versión 2.1, por lo que es necesario crear un ambiente para tal versión como está descrito en la sección de anexos.



## Instalación de TFX en un ambiente TensorFlow < 2.1
Al instalar TFX via pip

`!pip install tensorflow`

`!pip install tfx`

en un ambiente TensorFlow 2.0 se generan los siguientes errores:

```
ERROR: tensorflow-serving-api 2.1.0 has requirement tensorflow~=2.1.0, but you'll have tensorflow 2.0.0 which is incompatible.
ERROR: tensorflow-data-validation 0.21.2 has requirement avro-python3!=1.9.2.*,!=<2.0.0,>=1.8.1; python_version >= "3.0", but you'll have avro-python3 1.9.2.1 which is incompatible.
ERROR: ml-metadata 0.21.1 has requirement tensorflow!=2.0.*,<3,>=1.15, but you'll have tensorflow 2.0.0 which is incompatible.
ERROR: Cannot uninstall 'PyYAML'. It is a distutils installed project and thus we cannot accurately determine which files belong to it which would lead to only a partial uninstall.
```    

# Panorama general de los componentes de TFX
Diagrama de flujo de datos de los componentes de TFX
![TFX Components](images/tfx-components.png)
## TFX Pipeline
Define un flujo de datos a través de múltiples componentes, con la meta de implementar una tarea específica de machine learning (p.e. constuir y desplegar un modelo de regresión con datos específicos). Los componentes Pipeline se construyen con librerías TFX. El resultado de un pipeline es un objetivo de despliegue TFX o un servicio de una petición de inferencia.
## Artifactos
En un pipeline, un artifacto (**artifact**) es una unidad de datos que se comunica entre componentes. Generalmente los componentes tienen al menos un artifacto de entrada y un artifacto de salida. Todos los artifactos requieren tener metadatos (*metadata*) asociados que definen el tipo (**type**) y propiedades (**properties**) del artifacto. Los artifactos tienen que tener un tipo de artifacto definido en el repositorio metadatos de Machine Learning. 

El tipo de artifacto (**artifact type**) tiene un nombre único y una esquema de propiedades de sus instancias. TFX utiliza el tipo de artifacto como para determinar el uso del artifacto por componentes en el pipeline, no necesariamente para determinar el contenido físico del artifacto en el sistema de archivos.

Ejemplos de tipos de artifactos:
- Example: representa ejemplos materializados en TFRecord del buffer de protocolo `tensorflow::Example`, CSV, JSON o cualquier otro formato físico. Sin importar el formato físico, la forma en la que los Examples son empleados en un pipeline es exactamente la misma: se analizan para generar estadísticas, se validan contra un esquema esperado, se preprocesan en preparación para entrenamiento, se proveen a un Trainer para entrenar modelos, entre otras.
- Model: Puede representar un modelo entrenado exportado a diversos formatos físicos tales como TensorFlow SavedModel, ONNX, PMML o PKL (varios tipos tipos objetos Python que representan modelos). En todo caso, los modelos siempre se evalúan, analizan y despliegan para servir en pipelines.

ML Metadata define un conjunto de propiedades de artifacto que permiten distinguir las posibles variantes de un mismo tipo de artifacto. Por ejemplo, format puede ser una posible propiedad (artifact property) del artifacto Examples cuyos valores pueden ser `TFRecord`, `JSON`, `CSV`, entre otros.

En síntesis, **artifact type** define la ontología de artifactos en un sistema completo de TFX, mientras que **artifact properties** definen la ontología específica para un artifact type.

# Componentes de TFX Pipeline
Un TFX pipeline es una secuencia de componentes que implementan un pipeline de machine learning que está diseñado específicamente para tareas de machine learning escalables y de alto rendimiento. Esto incluye modelado, entrenamiento, servir inferencias, administrar despliegues a destinos en línea, móvil nativo y JavaScript.

Un TFX pipeline típicamente incluye los siguientes componentes:

|Componente|Propósito|
|:---|:--|
|ExampleGen|Componente inicial de entrada de un pipeline que recibe y opcionalmente particiona el dataset|
|StatisticsGen|Calcula las estadísticas inciales del dataset|
|SchemaGen|Examina las estadísitcas y crear el esquema de datos|
|ExampleValidator|Identifica anomalías y valores faltantes en el dataset|
|Transform|Lleva a cabo ingeniería de características en el dataset|
|Trainer|Entrena el modelo|
|Evaluator|Lleva a cabo un análisis detallado de los resultados de entrenamiento|
|ModelValidator|Ayuda a validar los modelos exportado, asegurando que son suficientemente buenos para ser desplegados en producción|
|Pusher|Despliega el modelo en una infraestructura de servicio|

Los componenentes están integrados por tres elementos: 
- Driver y Publisher: Provee de metadatos al Executor. Útil para depuración.
- Executor: Donde un componente lleva a cabo su procesamiento. Los desarrolladores escriben código que corre en el executor, de acuerdo con los requerimientos de las clases que implementan el tipo de componente con el que se está trabajando. Por ejemplo, al trabajar con el componente `Transform` es necesario desarrollar una función `preprocessing_fn`.

# Librerías TFX
TFX provee de librerías y componentes pipeline. Este diagrama ilustra la relación entre ellos.

![TXF Libraries](images/tfx-libraries.png)

TFX provee de diversos paquetes de Python que son las liberías que se emplean para crear componentes pipeline de tal forma que el código escrito por el desarrollador pueda enfocarse en los aspectos únicos de su implementación en el contexto del pipeline.

Las librerías TFX incluyen:

|Librería|Descripción|
|:---|:---|
|TensorFlow Data Validation (TFDV)|Para analizar y validar datos de machine learning. Diseñada para ser altamente escalable y trabajar bien con TensorFlow y TFX.|
|TensorFlow Transform (TFT)|Para preprocesar datos con TensorFlow. Útil para datasets que requieren de tratamiento integral en normalización, conversión de cadenas de caracteres a enteros, conversión de punto flotante a enteros (buckets).|
|TensorFlow|Empleado para entrenar modelos con TFX. Recibe datos de entrenamiento y código de modelado y crea un resultado `SavedModel`. También integra un pipeline de ingeniería de características creado por TensorFlow Transform para preprocesar datos de entrada.|
|TensorFlow Model Analysis (TFMA)|Librería para evaluar modelos de TensorFlow. Se usa junto con TensorFlow para crear un `EvalSavedModel` que se convierte en la base para su análisis. Permite a los usuarios evaluar sus modelos con grandes cantidades de datos de forma distribuida empleando las mismas métricas definidas en el entrenador. Estas métricas pueden ser calculadas sobre distintas fracciones de datos y visualizada en notebooks de Jupyter.|
|TensorFlow Metadata (TFMD)|Provee de representaciones estándar para metadatos que son útiles durante el entrenamiento de modelos de machine learning con TensorFlow. Los metadatos pueden ser producidos manualmente o automáticamente durante el análisis de datos de entrada, y pueden ser consumidos para validaciones de datos, exploración, transformaciones.
|ML Metadata (MLMD)|Librería para registrar y recuperar metadatos relacionados con flujos de trabajo machine learning de desarrolladores y científicos de datos.|

## Librerías de soporte
Requerida
- **Apache Beam** es un modelo unificado de código abierto para definir pipelines de procesamiento batch y streaming. TFX usa Apache Beam para implementar pipelines de datos en paralelo. El pipeline es ejecutado por uno de los backends de procesamiento distribuidos soportados por Beam que incluyen Apache Flink, Apache Spark, Google Cloud Dataflow, y otros.

Opcionales
- **Apache Airflow** plataforma para crear, calendarizar y monitorear flujos de trabajo de forma programática. Los flujos de trabajo definidos como código con más fáciles de mantener, controlar versiones, probar y colaborar.
- **Kubeflow** dedicada a hacer despliegues de flujos de trabajo de machine learning en Kubernetes simple, portable y escalable.
